# ModelV2
## Object detection non-overlapping cards

In [1]:
!pip -q install ultralytics wandb


In [2]:
from ultralytics import YOLO
import wandb

Enable W&B logging for Ultralytics in CLI

`yolo settings wandb=True`

Disable Weights & Biases logging

`yolo settings wandb=False`

In [7]:
wandb.login()

True

## Model
We can use a singel YOLO model for both, object detection and classification tasks.

In [9]:
model = YOLO("yolo11n.pt") # YOLOv11 nano for quick testing. Later we choose the larger model for better accuracy

## Train
imgsz: yolo resizes the input images to a square image. Since jasscards are not square, yolo resizes the long side to 640 and then scales the shorter side proportionally.

In [10]:
# Hyperparameters
epochs=50
imgsz=640 # 640 (default) – Balanced performance; 512 – Faster, lower memory usage, 1280+ – Best accuracy for small details but slower.
iterations=100 # Number of tuning cycles (higher = better tuning). The model tries different values for lr, momentum, weight decay, etc.

In [11]:
wandb.init(
    project="jassair",  # Specify project
    name="test_run_1", # Run name
    config={                 # Track hyperparameters and metadata
        "epochs": epochs,
        "imgsz": imgsz,
    },
)

When you choose iterations, the model tries to tune itself. Or you can tune them manually like below

In [ ]:
# Model with all tunable hyperparameters
model.train(
    data="path/to/data.yaml",
    epochs=epochs,
    imgsz=imgsz,
    batch=16,   # Number of images per batch
    lr0=0.01,   # Initial learning rate
    momentum=0.937,  # SGD momentum
    weight_decay=0.0005,  # Regularization
    optimizer="SGD",  # or Adam, AdamW
    hsv_h=0.015,  # Hue augmentation
    hsv_s=0.7,  # Saturation augmentation
    hsv_v=0.4,   # Value augmentation
    project="jassair", # Wandb project name
    name="test_run_1" # Wandb run name
)

In [14]:
wandb.finish()

In [13]:
results = model("path/to/image.jpg")
results.show()  # Display results


FileNotFoundError: path/to/image.jpg does not exist